In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import random
import math
import tqdm
import numpy as np
import pandas as pd

torch.manual_seed(192837)
rand = random.Random(192838)

In [2]:
CITY_COLUMN = 'city_ascii'
COUNTRY_COLUMN = 'iso2'

In [3]:
df_all = pd.read_csv('data/worldcities.csv')

'''
df_top_countries = df_all.groupby(COUNTRY_COLUMN).size().sort_values(ascending=False).reset_index()
classes = list(df_top_countries.head(N_CLASSES)[COUNTRY_COLUMN])
'''
classes = ['IN', 'US', 'BR', 'DE', 'CN', 'JP', 'RU', 'MX']
N_CLASSES = len(classes)

print(classes)
class_to_id = {classes[i]: i for i in range(N_CLASSES)}

df_all = df_all[df_all[COUNTRY_COLUMN].isin(classes)]
df_all = df_all[df_all[CITY_COLUMN].notna()]
df_all.head(10)

['IN', 'US', 'BR', 'DE', 'CN', 'JP', 'RU', 'MX']


,city,city_ascii,lat,lng,country,iso2,iso3,admin_name,capital,population,id
0,Tokyo,Tokyo,35.6870,139.7495,Japan,JP,JPN,Tōkyō,primary,37785000.0,1392685764
2,Delhi,Delhi,28.6100,77.2300,India,IN,IND,Delhi,admin,32226000.0,1356872604
3,Guangzhou,Guangzhou,23.1300,113.2600,China,CN,CHN,Guangdong,admin,26940000.0,1156237133
4,Mumbai,Mumbai,19.0761,72.8775,India,IN,IND,Mahārāshtra,admin,24973000.0,1356226629
6,Shanghai,Shanghai,31.2286,121.4747,China,CN,CHN,Shanghai,admin,24073000.0,1156073548
7,São Paulo,Sao Paulo,-23.5504,-46.6339,Brazil,BR,BRA,São Paulo,admin,23086000.0,1076532519
9,Mexico City,Mexico City,19.4333,-99.1333,Mexico,MX,MEX,Ciudad de México,primary,21804000.0,1484247881
10,Kolkāta,Kolkata,22.5675,88.3700,India,IN,IND,West Bengal,admin,21747000.0,1356060520
14,New York,New York,40.6943,-73.9249,United States,US,USA,New York,NaN,18832416.0,1840034016
15,Beijing,Beijing,39.9067,116.3975,China,CN,CHN,Beijing,primary,18522000.0,1156228865


In [4]:
character_set = set()
for city in df_all[CITY_COLUMN]:
    for ch in city:
        character_set.add(ch)

PAD_TOKEN = 0
tokens = ['<PAD>'] + sorted(list(character_set))
token_to_id = {tokens[i]: i for i in range(len(tokens))}
N_VOCAB = len(tokens)

''.join(tokens)

"<PAD> '()-./2ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz"

In [5]:
def encode(city: str, max_length: int):
    encoded = [token_to_id[city[i]] for i in range(min(max_length, len(city)))]
    return encoded + [PAD_TOKEN] * (max_length - len(encoded))

def decode(city_enc: list[int]):
    return ''.join([tokens[tk] for tk in city_enc if tk != PAD_TOKEN])

In [6]:
city_length_90_perc = np.quantile(np.array(list(map(len, df_all[CITY_COLUMN]))), 0.8)

SEQ_LENGTH = int(city_length_90_perc)
SEQ_LENGTH

12

In [7]:
data_list_x = [encode(city, SEQ_LENGTH) for city in df_all[CITY_COLUMN]]
data_list_y = [class_to_id[iso2] for iso2 in df_all[COUNTRY_COLUMN]]
data_list = list(zip(data_list_x, data_list_y))
rand.shuffle(data_list)

data_x = torch.tensor(np.array([x for x, _ in data_list], dtype=np.long))
data_y = torch.tensor(np.array([y for _, y in data_list], dtype=np.long))

for i in range(5):
    print(decode(data_x[i]), classes[data_y[i]])

TRAIN_TEST_SPLIT = 0.9
n_train = round(data_x.shape[0] * TRAIN_TEST_SPLIT)
train_x, train_y = data_x[:n_train, :], data_y[:n_train]
test_x,  test_y  = data_x[n_train:, :], data_y[n_train:]

print(data_x.shape[0], n_train, data_x.shape[0] - n_train)

Gohadi IN
Kandra IN
Chuanliaocun CN
Aracoiaba BR
Miryal IN
22320 20088 2232


In [8]:
class SimpleClassifier(nn.Module):
    def __init__(self,
                 emb_dim: int,
                 hidden_dim: int,
                 hidden_layers: int):
        super().__init__()

        self.emb = nn.Linear(N_VOCAB, emb_dim, bias=False)
        self.foot = nn.Linear(emb_dim * SEQ_LENGTH, hidden_dim)
        self.body = nn.ModuleList([nn.Linear(hidden_dim, hidden_dim) for _ in range(hidden_layers)])
        self.head = nn.Linear(hidden_dim, N_CLASSES)
        self.relu = nn.ReLU(inplace=True)
    
    def forward(self, x: torch.Tensor):
        if len(x.shape) == 2:  # unbatched input
            x = x.unsqueeze(0)
        B, *_ = x.shape

        x = self.emb(x).reshape((B, -1))
        x = self.relu(self.foot(x))
        for l in self.body:
            x = self.relu(l(x))
        x = self.head(x)

        return x

In [9]:
def train_epoch(model: nn.Module,
                optimizer: optim.Optimizer,
                dataset_x: torch.Tensor,
                dataset_y: torch.Tensor,
                batch_size: int):
    model.train()
    
    loss_sum = 0
    accu_sum = 0
    n_batches = math.ceil(dataset_x.shape[0] / batch_size)

    for i in tqdm.tqdm(range(n_batches), 'train'):
        x, y = dataset_x[i * batch_size: (i+1) * batch_size, :], dataset_y[i * batch_size: (i+1) * batch_size]
        x = F.one_hot(x, N_VOCAB).to(dtype=torch.float32)
        y_hat = model.forward(x)

        loss = F.cross_entropy(y_hat, y)
        loss_sum += loss.detach().clone()

        with torch.no_grad():
            accu_sum += torch.sum(y_hat.argmax(dim=-1) == y) / x.shape[0]

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
    
    return loss_sum / n_batches, accu_sum / n_batches

def test_epoch(model: nn.Module, dataset_x: torch.Tensor, dataset_y: torch.Tensor, batch_size):
    model.eval()
    
    with torch.no_grad():
        loss_sum = 0
        accu_sum = 0
        n_batches = math.ceil(dataset_x.shape[0] / batch_size)

        for i in tqdm.tqdm(range(n_batches), ' test'):
            x, y = dataset_x[i * batch_size: (i+1) * batch_size, :], dataset_y[i * batch_size: (i+1) * batch_size]
            x = F.one_hot(x, N_VOCAB).to(dtype=torch.float32)
            y_hat = model.forward(x)

            loss = F.cross_entropy(y_hat, y)
            loss_sum += loss
            accu_sum += torch.sum(y_hat.argmax(dim=-1) == y) / x.shape[0]
        
        return loss_sum / n_batches, accu_sum / n_batches

In [10]:
model = SimpleClassifier(emb_dim=16,
                         hidden_dim=32,
                         hidden_layers=6)
optimizer = optim.Adam(model.parameters(), lr=1e-3, weight_decay=5e-4)
N_EPOCHS = 50
BATCH_SIZE = 32

In [11]:
for epoch_i in range(N_EPOCHS):
    print(f'=== epoch {epoch_i} ===')

    train_loss, train_accu = train_epoch(model, optimizer, train_x, train_y, BATCH_SIZE)
    test_loss, test_accu = test_epoch(model, test_x, test_y, BATCH_SIZE)

    print('train loss:', train_loss.item())
    print(' test loss:', test_loss.item())
    print('train accu:', train_accu.item())
    print(' test accu:', test_accu.item())

=== epoch 0 ===


 test: 100%|██████████| 70/70 [00:00<00:00, 358.56it/s]


train loss: 1.6578563451766968
 test loss: 1.5767815113067627
train accu: 0.4182092547416687
 test accu: 0.47083330154418945
=== epoch 1 ===


 test: 100%|██████████| 70/70 [00:00<00:00, 227.22it/s]


train loss: 1.4602901935577393
 test loss: 1.4780000448226929
train accu: 0.4997677505016327
 test accu: 0.48020830750465393
=== epoch 2 ===


...

train loss: 0.4990904927253723
 test loss: 1.0830504894256592
train accu: 0.8353403806686401
 test accu: 0.6889880895614624
=== epoch 48 ===


 test: 100%|██████████| 70/70 [00:00<00:00, 1021.13it/s]


train loss: 0.4950273334980011
 test loss: 1.0912714004516602
train accu: 0.8378118276596069
 test accu: 0.6913690567016602
=== epoch 49 ===


 test: 100%|██████████| 70/70 [00:00<00:00, 923.34it/s]

train loss: 0.4871978759765625
 test loss: 1.0959736108779907
train accu: 0.8391056060791016
 test accu: 0.6895833015441895


In [12]:
for i in range(20):
    print(decode(test_x[i]), classes[test_y[i]], end=' ')
    y_hat = torch.softmax(model.forward(F.one_hot(test_x[i], N_VOCAB).to(dtype=torch.float32)), dim=-1).squeeze()
    print(classes[y_hat.argmax(dim=-1)], f'({y_hat.max().item() * 100:.2f}%)')

Leme BR BR (38.76%)
Minatitlan MX MX (63.12%)
Valley US US (92.83%)
Garden City US US (99.52%)
Hampton US US (77.23%)
Dicholi IN IN (72.40%)
Pau Brasil BR BR (97.84%)
Taloda IN US (60.19%)
Mutum BR US (25.32%)
Degana IN IN (58.53%)
Sao Goncalo  BR BR (99.01%)
Pileru IN IN (66.45%)
Hunsur IN DE (27.55%)
Vincennes US DE (81.73%)
Lawrence US US (96.06%)
Bad Iburg DE US (46.03%)
Morbi IN IN (46.55%)
Othello US DE (47.44%)
Amari IN IN (77.16%)
Lobau DE US (40.57%)


In [13]:
from torch.distributions import Categorical
from torch.distributions.gumbel import Gumbel
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts

class AntiEmb():
    def __init__(self,
               inner_net: nn.Module,
               desired_c: int,
               lr: float=1e-2,
               right_c_coef: float=1.0,
               wrong_c_coef: float=1.0):
        super().__init__()

        self.input_shape = (1, SEQ_LENGTH, N_VOCAB)
        self.input = nn.Parameter(
            data=torch.normal(0, 1, size=self.input_shape),
            requires_grad=True
        )

        self.inner_net = inner_net
        self.grad_eraser = torch.optim.SGD(self.inner_net.parameters())
        self.anti_optimizer = torch.optim.Adam((self.input,), lr=lr)
        # self.anti_scheduler = CosineAnnealingWarmRestarts(self.anti_optimizer, T_0=50, T_mult=2)

        self.desired_c = desired_c
        self.right_c_coef = right_c_coef
        self.wrong_c_coef = wrong_c_coef

        # self.input_min = 0.0
        # self.input_max = 1.0

        self.gumbel = Gumbel(loc=0.0, scale=1.0)
        self.epsilon = 1e-8

    def test(self):
        with torch.no_grad():
            return self.inner_net.forward(self.input)

    def zero_grad(self):
        self.grad_eraser.zero_grad()
        self.anti_optimizer.zero_grad()

    def crop(self):
        self.input.data.copy_(torch.clamp(self.input.detach().clone(), self.input_min, self.input_max))
    
    def step(self):
        prev_input = self.input.detach().clone()

        probs = torch.softmax(self.input, dim=-1)
        gumb = self.gumbel.sample(self.input_shape)

        j_onehot = F.one_hot(torch.argmax(gumb + torch.log(probs + self.epsilon), dim=-1), N_VOCAB)
        j_onehot = j_onehot.to(torch.float32).requires_grad_(True)

        j_continuous = torch.softmax(gumb + torch.log(probs + self.epsilon), dim=-1)

        output = self.inner_net.forward(j_onehot).squeeze()
        output_soft = torch.softmax(output, dim=-1)
        
        loss = output_soft[torch.arange(0, N_CLASSES) != self.desired_c].max() * self.wrong_c_coef + \
               output_soft[self.desired_c] * -self.right_c_coef
        loss.backward()

        j_continuous.backward(j_onehot.grad)

        self.anti_optimizer.step()
        self.zero_grad()

        # self.anti_scheduler.step()

        # self.crop()

        return prev_input, output.detach().clone(), loss.detach().clone()

In [14]:
ANTI_NET_STEPS = 3000
TEST_TRIES = 5

for i in range(N_CLASSES):
    desired_c = i

    print(f'-=-=- step {i} -=-=-')
    print(f'des class:', classes[desired_c], f'({desired_c})')

    fin_loss = float('+inf')
    fin_inp, fin_outp = None, None

    for try_i in range(TEST_TRIES):
        anti_net = AntiEmb(model, desired_c, lr=5e-2)

        for step_i in tqdm.tqdm(range(ANTI_NET_STEPS), f'attempt #{try_i}'):
            inp, outp, loss = anti_net.step()
        
        if loss.item() < fin_loss:
            fin_loss = loss.item()
            fin_inp = inp
            fin_outp = outp
        
        print(f'attempt #{try_i} loss:', loss.item())
    
    print(f'fin loss:', fin_loss)
    print(f'fin output (from function):', fin_outp)

    inp_argmax = fin_inp.argmax(dim=-1)

    print(f'fin input  (argmax):', decode(inp_argmax.squeeze()))
    print(f'fin output (argmax):', model.forward(F.one_hot(inp_argmax, N_VOCAB).to(torch.float32)))

    dist = torch.distributions.Categorical(logits=fin_inp)
    sample = dist.sample()
    entropy = dist.entropy()

    print(f'fin input  (sample):', decode(sample.squeeze()))
    print(f'fin output (sample):', model.forward(F.one_hot(sample, N_VOCAB).to(torch.float32)))
    print(f'fin input entropy:', entropy)

-=-=- step 0 -=-=-
des class: IN (0)


attempt #0:   0%|          | 0/3000 [00:00<?, ?it/s]

attempt #0: 100%|██████████| 3000/3000 [00:28<00:00, 105.26it/s]


attempt #0 loss: -1.0


attempt #1: 100%|██████████| 3000/3000 [00:21<00:00, 140.94it/s]


...

attempt #2 loss: -0.8908573389053345


attempt #3: 100%|██████████| 3000/3000 [00:11<00:00, 268.05it/s]


attempt #3 loss: -0.8882911801338196


attempt #4: 100%|██████████| 3000/3000 [00:14<00:00, 202.28it/s]


attempt #4 loss: -0.87801593542099
fin loss: -0.8908573389053345
fin output (from function): tensor([-1.0934,  2.6656,  2.4943, -2.4017, -5.1932, -5.7134, -8.7659,  5.8622])
fin input  (argmax): sqcFaYAIqi F
fin output (argmax): tensor([[-1.0934,  2.6656,  2.4943, -2.4017, -5.1932, -5.7134, -8.7659,  5.8622]],
       grad_fn=<AddmmBackward0>)
fin input  (sample): sqcFuYAIqi F
fin output (sample): tensor([[-1.0831,  2.6334,  2.5211, -2.4175, -5.1583, -5.6532, -8.7498,  5.8585]],
       grad_fn=<AddmmBackward0>)
fin input entropy: tensor([[0.9058, 0.0627, 0.2960, 0.4487, 0.7257, 0.1034, 0.5485, 0.0823, 0.2642,
         0.1631, 0.2633, 0.0858]])
